<a href="https://colab.research.google.com/github/davidnoone/PHYS332_FluidExamples/blob/main/08_Convection_FV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thermal (Rayleigh-Benard) convection in 2d

We wish to consider a problem of thermal convection using the non-divergent euler equations in 2d. We can formulate the problem terms of vorticity, and uses simple numerical methods to do so.

For an nearly incompressible invisid fluid that is near hydrostatic (i.e., like water, or air), the governing equations are:

$$
\frac{d u}{d t} = -\frac{\partial \Phi}{\partial x} 
$$
$$
\frac{d v}{d t} = -\frac{\partial \Phi}{\partial y} + b 
$$

Where the buoyancy, b, is defined

$$
b = g \frac{\theta}{\theta_0}
$$

Being related to entropy via the potential temperature, $\theta$, b is conserved, but may have an external source (e.g., such as heating)
$$
\frac{d b}{d t} = Q
$$


With incompressibility, density is assumed constant, and mass continuity is just a statement that the divergence is zero.
$$
  \frac{\partial u}{dx} + \frac{\partial v}{dy} = 0
$$

This immediately allows us to define a streamfunction as the only quantity needed to describe the velocity field. That is:

$$ 
    u = - \frac{\partial \Psi}{\partial y}
$$

$$
    v = \frac{\partial \Psi}{\partial x}
$$

The vorticity is related to the streamfunction as: 

$$
  \zeta = \frac{\partial^2 \psi}{\partial x^2} 
        + \frac{\partial^2 \psi}{\partial y^2}
$$


The momentum can be described entirely by predicting the evolution of vorticity. Taking the curl of the u and v equations we find the pressure terms cancell leading to a rearkably elegant prognostic equation for voticity:
$$
\frac{d \zeta}{d t} = 
  \frac{\partial}{\partial x}(\frac{d v}{d t}) 
  - \frac{\partial}{\partial y}(\frac{d u}{d t})
$$

We may note that the total derivative can be expanded to find the advective terms for each quantity "A"
$$
 \frac{d A}{dt} = \frac{\partial A}{\partial t} +
    u \frac{\partial A}{\partial x} + v \frac{\partial A}{\partial y}
$$

Here "y" is considered to be the vertical, such that gravity points downward in the y coordinate. Since the flow is incompressible, the $\rho$ is constant, and this second form becomes particularly convinient for numerical methods. 

With the non-divergent condition, the advection term can be written in a conservative form: 

$$
  u \frac{\partial A}{\partial x} + v \frac{\partial A}{\partial y} = 
  \frac{\partial (uA)}{\partial x} + \frac{\partial (vA)}{\partial y}   
$$

The final Eulerian prognostic model is therefore described with two  equations that describe conservation with the the addition  of buoyancy which can induce rotation:


$$
\frac{\partial \zeta}{\partial t} = 
                      - \frac{\partial (u \zeta)}{\partial x}
                      - \frac{\partial (v \zeta)}{\partial y}
                      + \frac{\partial     b}{\partial x}
$$
And
$$
\frac{\partial b}{\partial t} = 
                      - \frac{\partial (ub)}{\partial x}
                      - \frac{\partial (vb)}{\partial y} + Q
$$



## Solution scheme
From the above, it should be clear that the eevolution of this system can be developed as follows:

  1. Begining with with the $\zeta^n$ and $b^n$ at some initial time ($\zeta$ could be calculated if u and v are known, of course)
  2. Solve the poisson equation to evaluate the streamfunction from the known vorticity. i.e., obtain $\psi^n$
  3. Caluclate the velocities $(u,n)^n$ as the gradients in $\psi^n$
  4. Use the velocities to calculate the advection of $\zeta$ and b. 
  5. Compute the x-gradient of b to add contributions to the acceleration due to buoyancy. At this stage a (viscous) diffusion term can also be added. (Not shown algebraically)
  6. Finally, advance $\zeta$ and b in time: step from time $n$ to $n+1$

This scheme can be repeated until the desired integration in time is complete.


# Solution method

Finite difference methods can be used to evaluate gradients. The poisson equation is solved using an iterative method that applies the finite difference analog of the Laplacian in 2d.  If advection is evaluated using finite difference methods, a substantial amount of diffusion is needed to stabilize the numerical result. This is possible, however, a more accurate scheme can also be used with better results.The stepping in time is done using a 3rd order scheme, rather than a simple single forward step, because it has some nice stability properties. This is accomplished using the weighted result from three applications of a simple forward Euler time step.





In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

print("Modules imported.")

Some run time initialization

In [ ]:
# Some constants
gravity = 9.81    # acceleration due to gravity
theta0  = 300.    # reference potential temperature [Kelvin]

# dimensions
nx = 30          # must be even
ny = 30          # even or odd

dx = 1.0        # grid spacing [metres]
dtime = 0.5     # time step [seconds]

nelapse = 50    # number of steps to run
#nelapse = 200    # number of steps to run
nskip = 5       # steps between plotting output
nstep = 0

xmax = nx*dx
ymax = ny*dx
xmid = np.linspace(0,xmax,nx)
ymid = np.linspace(0,ymax,ny)

# Boundary conditions: top and bottom streamfunction, and buoyancy flux
psi_top = 0.0 
psi_bot = 0.0

bpert = 0.001*gravity/theta0   # Random buoyancy perturbation b = g * theta/theta0
bflux = 0*0.010*gravity/theta0   # units "per second"            # {TRY THIS}
bbubl =   1.000*gravity/theta0 # add a bubble                  # {TRY THIS}

kdiff = 0.*dx*dx/50.            # [Optional] diffusivity m2/sec (little bit for stability)


# Define the working arrays here to make them global (mostly so we can plot them)
vort = np.zeros((ny,nx)) 
buoy = np.zeros((ny,nx)) + bpert*np.random.randn(ny,nx)
psi  = np.zeros((ny,nx))
uwnd = np.zeros((ny,nx))
vwnd = np.zeros((ny,nx))

# Initialize a hot bubble
def add_bubble(xpos, ypos, width):
  xx,yy = np.meshgrid(xmid,ymid)
  dist = np.sqrt((xx-xpos)**2 + (yy-ypos)**2)/width
  bone = np.exp(-dist**2)     # gaussian
  return bone

width = 4.0
bone = add_bubble(xmax/2.,ymax/3,4)
buoy = buoy + bone*bbubl 

print('Initialization complete.')

A collection of finite difference opperators needed later on. Use the numpy "roll" function to impliment periodic boundary conditions, but explictly use one-sided derivatives in the "y" direction at boundary edges.

In [ ]:
# Centered finite difference X derivative (2nd order), with periodic BCs: dF/dx
def fd_dfdx(fld,dx):
    dfdx = 0.5*(np.roll(fld,-1, axis=1) - np.roll(fld,+1, axis=1))/dx
    return dfdx

#Centered finite difference in Z with single sided ends: dF/dy
def fd_dfdy(fld,dx):
    global ny
    dfdy = np.zeros_like(fld)
    dfdy[0     ,:] = (fld[1     ,:] - fld[     0,:])/dx
    dfdy[1:ny-1,:] = (fld[2:ny-0,:] - fld[0:ny-2,:])/(2*dx)
    dfdy[  ny-1,:] = (fld[  ny-1,:] - fld[  ny-2,:])/dx
    return dfdy 

# Laplacian: del^2(F)
def fd_lap(fld,dx):
    flap =  -4*fld + np.roll(fld,-1, axis=1) + np.roll(fld,+1, axis=1)
    flap[0     ,:] = flap[0     ,:] + (fld[1     ,:]                )
    flap[1:ny-1,:] = flap[1:ny-1,:] + (fld[2:ny-0,:] + fld[0:ny-2,:])
    flap[  ny-1,:] = flap[  ny-1,:] + (                fld[  ny-2,:])
    return flap

# Curl/vorticity: del x (u,v) [Same as fd_div(v,-u)]
def fd_curl(u, v, dx):
    return fd_dfdx(v,dx) - fd_dfdy(u,dx)

# Divergence: del.(u,v)
def fd_div(u, v, dx):
    return fd_dfdx(u,dx) + fd_dfdy(v,dx)

# Advection opperator using finite differences [not very accurate]
def advect(fld,u,v,dx):
    return -1.0*(u*fd_dfdx(fld,dx) + v*fd_dfdy(fld,dx))


## Advection using finite volume methods

A finite volume method is used to evaluate the (non-linear) advection. Finite volume methods have some appealing mass conservation properties, and they can be configured to help prevent noisy solutions that come about from low order approximations to the continuous equations. This is especially important for managing non-linear terms.

Here we use a second order scheme where the advection tendency is computed as

$$
\left( \frac{\partial \zeta}{\partial t}\right)_n = 
  \frac{1}{\Delta x} \left( F_{i+1/2} - F_{i-1/2} \right)
$$

The fluxes $F$ are at the interfaces (locations at "half" levles), and which must be estimated by linear reconstruction. The details of the estimation gives rise to a robust and accurate scheme. Note that an estimate of the value at each interface is can be obtained from either side. That is, the left side of the interface at $i-1/2$ can be estimated using values from the left or from the right side. Consider the two options:

$$
F_{L,i-1/2} = F_{i-1} + \frac{1}{2}\Delta F_{1-1/2}
$$
$$
F_{R,i-1/2} = F_{i} - \frac{1}{2}\Delta F_{1+1/2}
$$

The half here emerges from wanting to extrapolate half a grid length, $\Delta x$, to get from the cell center to the cell interfaces.
The gradients $\Delta F$ can adjusted to ensure monotonicity (and piece wise linear), which equivieltly ensures that no new oscillations are produced in the solution. The slope is set to be no larger (in magnitude) than estimates of $\Delta F$ from the upstream, downstream and centered pairs of values of $\zeta$ that can from the possible combinations in the range from $i-1$ to $i+1$. (_There are a range of specific limiting functions, the "minmod" method used here is simple to impliment and works well._)


Finally, the flux is estimated as the mean of these two estimates. The mean takes a special form involving a correction associate with the maximum possible (wave) speed supported by the equations of motion. This correction term effectivivly stabilizes the solution.

$$
  F = \frac{1}{2} \left[ (F_L + F_R) + \alpha (\zeta_R - \zeta_L) \right]
$$

where $\alpha = max(u_L, u_R)$. More formally, $\alpha$ is the Jacobian of the flux $\partial F/\partial u$. If we were to select $\alpha = 0$, we would identically recover the a centered finite difference method!

Using these fluxes for each interface the advection tendency is obtained.




In [ ]:
# Two-D advection using a 2nd order FV method
R = -1   # right
L = 1    # left

# Piecewise linear reconstruction of interface values, using a minmod slope limiter
def reconstruct(f, dx, axis):
    # Compute limited slope/gradient
    f_dx = 0.5*( np.roll(f,R,axis=axis) - np.roll(f,L,axis=axis) )   # centered difference
    f_dx = np.maximum(0., np.minimum(1., ( (f-np.roll(f,L,axis=axis)))/(f_dx + 1.0e-8*(f_dx==0)))) * f_dx
    f_dx = np.maximum(0., np.minimum(1., (-(f-np.roll(f,R,axis=axis)))/(f_dx + 1.0e-8*(f_dx==0)))) * f_dx

    # get left and right side estimates of the fluxes
    f_R = f + 0.5*f_dx
    f_L = f - 0.5*f_dx
    f_L = np.roll(f_L,R,axis=axis)
    return f_L, f_R

# Form the fluxes: Lax-Freidrichs type scheme
def advect_flux_1d(f, u, dx, axis):
    u_L, u_R = reconstruct(u, dx, axis)
    f_L, f_R = reconstruct(f, dx, axis)
    alpha = np.maximum( np.abs(u_L), np.abs(u_R) )
    flx = 0.5*((u_L*f_L + u_R*f_R) - alpha*(f_L - f_R))
    return flx

# Evaluate advective tendency as divegence of (interface) fluxes
def advect_tend(f,u,v,dx):
    dy = dx
    flx_x = advect_flux_1d(f, u, dx, 1)
    flx_y = advect_flux_1d(f, v, dy, 0)

    fadv = (np.roll(flx_x,L,axis=1) - flx_x)/dx + \
           (np.roll(flx_y,L,axis=0) - flx_y)/dy

    return fadv


## Inverting $\zeta$: a Poisson equation

Finding $\psi$ from $\zeta$ requires solution of the poisson equation

$$
  \nabla^2 \psi = \zeta
$$

Accomplishing this is one of the main numerical tasks in this calculation. Solving this equation effectly solves for mass continuity by defining a streamfunction. 

Using finite differences we have:

$$
  \frac{
        \left( \psi_{i+1,j} - 2\psi_{i,j} + \psi_{i-1,j} \right) 
      }{(\Delta x)^2} 
 + \frac{
      \left( \psi_{i,j+1} - 2\psi_{i,j} + \psi_{i,j-1} \right)
      }{(\Delta y)^2}  
  = \zeta_i
$$

Let's assume $\Delta x = \Delta y$, and rearrange to obtain:

$$
  \psi_{i,j}  = \frac{1}{4} \left(
    \psi_{i-1,j} + \psi_{i+1,j} + \psi_{i,j-1} + \psi_{i,j+1}
    - \frac{\zeta_{i,j}}{(\Delta x)^2}
    \right)
$$

This result suggest we can compute an updated estimate of $\psi_{i,j}$ knowing the value of $\zeta_{i,j}$, and the values of $\psi$ nearby. 

Notice the detail that for i and j as odd values, the right hand side has only even values (i+1, i-1, j+1, and j-1). One can imagine the red and black squares on a chess board. Therefore this equation can be solved for the black squares, needing only the red squares, then  solved again for the red squares needing only the black squares. Indeed, the approach can be repeated to obtain increasingly better estimates of $\psi$. Ususually numerous iterations are required to obtain the needed accuracy. This approach is called a relaxation method, and described as the Gauss-Seidel method.

The method can be accelerated by introducing an "over relaxation" in which we take the view that any updated estimate is still likely to be an underestimate, so multiply the size of the update by some factor. 
That is, 

$$
  \psi_{i,j}^*  = (1-w)\psi_{i,j} + \frac{w}{4} \left(
    \psi_{i-1,j} + \psi_{i+1,j} + \psi_{i,j-1} + \psi_{i,j+1}
    - \frac{\zeta_{i,j}}{(\Delta x)^2}
    \right)
$$

Where the * denotes the updated estimate and the parameter w is tuned to accelerate the convergence in the range $1 \le w \lt 2$. The case of $w=1$ has no accelaration and larger numers are faster, but can be unstable for noisy fields. Some care is needed in selecting it.

Finally, the convergence is faster if we start with a good guess! One good guess is the value from the previous model timestep. While not implimented here, another good guess is from simple linear extrapolation in time.

$$
  \psi_{guess}^{n+1} = 2\psi^{n} - \psi^{n-1}
$$



In [ ]:
# initialize red/black indices
(jm, im) = np.meshgrid(np.arange(ny),np.arange(nx))
x = np.zeros((nx,ny),dtype=np.int)
x[:,1:-1] = im[:,1:-1]+jm[:,1:-1]
ir, jr = np.where((x>0) & (x%2 == 0))
ib, jb = np.where((x>0) & (x%2 == 1))

# set periodic boundary conditions in the "i" direction
irp = ir + 1
irp[irp>nx-1] -= nx
irm = ir - 1
irm[irm<   0] += nx

ibp = ib + 1
ibp[ibp>nx-1] -= nx
ibm = ib - 1
ibm[ibm<   0] += nx

# Solve possion equation using Red/Black Gauss-Siedel method
def psolve(psi_bot,psi_top,psi0,vor,dx):
    err_max = 1.0e-5      # acceptable fractional error. 
    niter_max = 200       # maximum allowable iterations (usually less)
    w = 1.5               # Over relaxation parameter (1 to 2)
    dxsq = dx*dx

    # Set first guess and boundary conditions
    psi = psi0
    psi[ny-1,:] = psi_top
    psi[   0,:] = psi_bot

    # Iterative solver by relaxation: red then black
    for niter in range(niter_max):
        psi_last = psi.copy()
        psi[jr,ir] = (1-w)*psi[jr,ir] + 0.25*w*(psi[jr,irp] + psi[jr,irm] + psi[jr+1,ir] + psi[jr-1,ir] - vor[jr,ir]*dxsq)
        psi[jb,ib] = (1-w)*psi[jb,ib] + 0.25*w*(psi[jb,ibp] + psi[jb,ibm] + psi[jb+1,ib] + psi[jb-1,ib] - vor[jb,ib]*dxsq)

        err = np.sum(np.abs(psi - psi_last))/(np.sum(np.abs(psi)) + 1.0e-18)
        if err < err_max:
            break

    if (niter >= niter_max):
        print("PSOLVE: Failed to converge (good luck!) err=",err)
        
    return psi



# Integrating the convection equations
The hard part is more or less done. We now just need to set up the main formation of the time derivatives ("tendencies"), following the numbered stratergy given above. Then, finaly, do the time stepping.



In [ ]:
def tendency():
    global vort, buoy, uwnd, vwnd, psi, dx, psi_bot, psi_top
    global dtime, nstep, nskip

    psi = psolve(psi_bot,psi_top,psi,vort,dx)
    uwnd = -fd_dfdy(psi,dx)                  
    vwnd =  fd_dfdx(psi,dx)                  

    ztend = advect_tend(vort,uwnd,vwnd,dx) + kdiff*fd_lap(vort,dx) + fd_dfdx(buoy,dx)
    btend = advect_tend(buoy,uwnd,vwnd,dx) + kdiff*fd_lap(buoy,dx)

    # Heating at bottom
    btend[   0,:] += bflux

    # Cooling at top
#    btend[ny-1,:] -= bflux

    # Cooling everyhere ("atmosphere")
    btend[:,:] -= bflux/ny


    return (ztend, btend)

# Use a 3rd order SSP RK scheme to integrate: F(n+1) = F(n) + dt*dF/dt
def advance(dtime): 
    global vort, buoy
    vort0 = vort
    buoy0 = buoy

    (ztend, btend) = tendency()
    vort = vort0 + dtime*ztend
    buoy = buoy0 + dtime*btend

    (ztend, btend) = tendency()
    vort = (vort0 + vort + dtime*ztend)/2.0
    buoy = (buoy0 + buoy + dtime*btend)/2.0

    (ztend, btend) = tendency()
    vort = (vort0 + 2.0*(vort + dtime*ztend))/3.0
    buoy = (buoy0 + 2.0*(buoy + dtime*btend))/3.0

    return


In [ ]:
#for nstep in range(ntime):
#    advance(dtime)

We will want to visualize the model state, define a function to plot the vorticity, buoyancy, and velocty field.

In [ ]:
def init_frame():
    cf.set_array([])
    cl.set_array([])
    return cl,cf,

def advance_frame(w):
    global vort, buoy, uwnd, vwnd, psi
    global dtime, nstep, nskip
    global cl, cf

    cno = np.max(np.sqrt(uwnd*uwnd + vwnd*vwnd))*dtime/(1.41*dx)

    # INTEGRATE THE EQUATIONS IN TIME: update to the next save time
    for n in range(nskip):
        nstep = nstep + 1
        if (cno > 2):
            print('CFL instability detected: abort! cno=',cno)
        advance(dtime)

    # Do the plot on the predefine plotting axes
    print('Frame: ',w,' nstep =',nstep, 'CFL=',cno)
    ax.set(title="Vorticity and streamfunction nstep="+str(nstep).zfill(4))
    for c in cf.collections:
        c.remove()
    cf = ax.contourf(xx,yy,buoy,cmap=colormap,levels=19)
    
    for c in cl.collections:
        c.remove()
    cl = ax.contour(xx,yy,psi,colors='black')

    return cl, cf,



To actually run the integration,  plot at various the needed time interval. 

In [ ]:
# Create an animation object, with frames created as the model runs
xx, yy = np.meshgrid(xmid,ymid)
colormap = 'jet'

fig = plt.figure(figsize=(6,6))
ax = plt.axes(xlim=(0, xmax), ylim=(0, ymax))  
ax.set(xlabel='X position [m]', ylabel='Y position [m]')
cf = ax.contourf(xx,yy,buoy,cmap=colormap,levels=19)
cl = ax.contour(xx,yy,psi,colors='black')

anim = animation.FuncAnimation(fig, advance_frame, init_func=init_frame, frames=nelapse, 
                                          interval=50,repeat=True)
plt.close()

print('Finished')






In [ ]:
# Run the model by instantiating the animation.
anim

In [ ]:
# playground


